# I. Algorithm

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

In [ ]:
# Step 1: Define the Euclidean distance function
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))

# Step 2: Implement the KNN class
class KNN:
    def __init__(self, k=3):
        self.k = k  # Number of neighbors

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = [self._predict(x) for x in X_test]
        return np.array(predictions)

    def _predict(self, x):
        # Compute the distance between x and all points in the training data
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
        
        # Get the indices of the k nearest neighbors
        k_indices = np.argsort(distances)[:self.k]
        
        # Get the labels of the k nearest neighbors
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        
        # Return the most common class label among the k neighbors
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

# Step 3: Data Preparation
# Example: Encoding the 'Material' column as 1 for wood and -1 for steel
#coaster_data['Material'] = coaster_data['Type'].apply(lambda x: 1 if 'Wood' in x else -1)

# Select features and target variable
#X = coaster_data[['height_ft', 'speed_mph', 'Gforce_clean']].values  # Features
#y = coaster_data['Material'].values  # Target (1 for Wood, -1 for Steel)

# Split the data manually (80% train, 20% test)
#split = int(0.8 * len(X))
#X_train, X_test = X[:split], X[split:]
#y_train, y_test = y[:split], y[split:]

# Step 4: Train the model using KNN
k = 5  # Number of neighbors
knn = KNN(k=k)
#knn.fit(X_train, y_train)
#
# Step 5: Make predictions on the test data
#y_pred = knn.predict(X_test)

# Step 6: Evaluate the model
def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

# Calculate accuracy
accuracy_score = accuracy(y_test, y_pred)
print(f'Accuracy: {accuracy_score * 100:.2f}%')

